#### Check installation and import libraries

In [ ]:
!copernicusmarine --version

In [ ]:
import copernicusmarine
import os
from datetime import datetime,timedelta, timezone
from src.copernicusmarine_Parser import BucketParser
from src.leaflet_maps import mymap, markers_from_geojson, get_ids, filter_markers, seed_from_geopandas
from ipywidgets import HTML
from IPython.display import Image
import geopandas as gpd
import logging
import numpy as np

# Configure the logging level to INFO
logging.basicConfig(level=logging.INFO)

## Set experiment parameters

In [ ]:
HTML("""<div style="display: flex; justify-content: center; transform: scale(0.8);width: 100%; margin: auto;">
    <iframe src="https://data.marine.copernicus.eu/product/SEAICE_ARC_SEAICE_L4_NRT_OBSERVATIONS_011_007/description" width="100%" height="800px"></iframe>
</div>""")

In [ ]:
# you may fix date here if need, eg. yyyy,mm,dd=(2025,5,27) 
date=datetime.now()
yyyy,mm,dd=date.year, date.month, date.day

## List, download and display Iceberg shapefile from Copernicus Marine

#### Loading catalog information for `cmems_sat-si_arc_berg-point_nrt_iw_d` dataset

In [ ]:
catalog=copernicusmarine.describe(contains=['cmems_sat-si_arc_berg-point_nrt_iw_d'])#,disable_progress_bar=True)
url=catalog.products[0].datasets[0].versions[0].parts[0].services[0].uri

#### Listing files from S3 bucket URL

In [ ]:
# Initialize BucketParser
s3 = BucketParser(url=url)

# List contents of the bucket with a pattern
items = s3.list_bucket(path=f"{yyyy:04d}/{mm:02d}",pattern=f"{yyyy:04d}{mm:02d}")

print("Data files listed:")
for i,item in enumerate(items): print(f"{i+1} - {item.split('/')[-1]}")

#### Download and extract latest shapefile data

In [ ]:
shp_file=s3.get([items[-1]])[0]+'/'

#### Read and display shapefile data

In [ ]:
for layer in gpd.list_layers(shp_file).name:
    if layer.startswith("iceberg"): break

In [ ]:
# Read the shapefile using geopandas
gdf = gpd.read_file(shp_file, layer=layer)
gdf

#### Selecting Icebergs on a map

In [ ]:
feature_collection=mymap(gdf)

#### Filter out icebergs from shapefile and display
* removing icebergs of size < 1000m^2

In [ ]:
filtered_feature=filter_markers(gdf, feature_collection, size=1000.)
iceberg_ids=get_ids(filtered_feature)
filtered_gdf = gdf.iloc[iceberg_ids]
filtered_collection=mymap(filtered_gdf,zoom=7,subsample=1)

## Run OpenBerg simulations on subset

#### Initialize OpenBerg model

In [ ]:
from opendrift.models.openberg import OpenBerg
from opendrift.readers.reader_netCDF_CF_generic import Reader

o = OpenBerg(loglevel=50) 
o.set_config('drift:vertical_profile', False) # use surface currents for this test
o.set_config('drift:horizontal_diffusivity', 100)

#### Adding readers to forcing data (currents, winds)
* using TOPAZ6 15 minutes Arctic forecasts for currents
* using NCEP Global Atmospheric Model (GFS) forecasts for winfds

In [ ]:
o.add_readers_from_list([
        'https://thredds.met.no/thredds/dodsC/cmems/topaz6/dataset-topaz6-arc-15min-3km-be.ncml',
        'https://pae-paha.pacioos.hawaii.edu/thredds/dodsC/ncep_global/NCEP_Global_Atmospheric_Model_best.ncd'])

#### Seeding virtual particles on iceberg positions
* seeding 1 particule per 50m^2 (with a minimum of 4 particles)
* simulating icebergs' drafts & sails ranging from 20 to 120m & 5 to 50m respectively

In [ ]:
seed_from_geopandas(o,filtered_gdf, time=datetime(yyyy, mm, dd-1))

#### Run simulations (over 2 days)

In [ ]:
simulation_days = 2 #Modify here for extending/shortening simulation
o.run(duration=timedelta(days=simulation_days))

## Show results

#### plot trajectories map

In [ ]:
o.plot(fast=True)

#### Display virtual icebergs animation

In [ ]:
o.animation(fast=True,filename='./simulation.gif',background=['x_sea_water_velocity', 'y_sea_water_velocity'],show_trajectories=True)
Image(url='./simulation.gif')

#### Show iceberg speed over time

In [ ]:
import matplotlib.pyplot as plt
iceberg_speed = np.sqrt(o.result.iceb_x_velocity**2 + o.result.iceb_y_velocity**2)
iceberg_speed.plot.line(x='time', add_legend=False, color='gray')
plt.ylabel('Iceberg speed  [m/s]')
plt.show()